In [1]:
import re
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pandas as pd
from IPython.display import display, clear_output
import ipywidgets as widgets

# First ensure widgets are enabled
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix

# Download required NLTK data
#nltk.download('vader_lexicon')

class FraudDetector:
    def __init__(self):
        print("Initializing fraud detector...")  # Debug print
        self.sia = SentimentIntensityAnalyzer()
        self.tax_evasion_keywords = ["pay cash", "no invoice", "avoid tax", "black money"]
        self.threat_keywords = ["pay or else", "leak your data", "harm you"]
        self.train_model()
    
    def train_model(self):
        print("Training model...")  # Debug print
        data = pd.DataFrame([
            {"text": "Pay $1000 or we leak your photos", "label": "threat"},
            {"text": "Let's deal in cash to avoid GST", "label": "tax_fraud"},
            {"text": "Hi, how are you?", "label": "safe"}
        ])
        self.vectorizer = TfidfVectorizer(max_features=1000)
        X = self.vectorizer.fit_transform(data['text'])
        self.model = LogisticRegression()
        self.model.fit(X, data['label'])
        print("Model trained successfully!")  # Debug print
    
    def analyze_message(self, text):
        print(f"Analyzing: {text}")  # Debug print
        # Threat detection
        threats_found = any(keyword in text.lower() for keyword in self.threat_keywords)
        text_vec = self.vectorizer.transform([text])
        ml_prediction = self.model.predict(text_vec)[0]
        
        # Tax fraud detection
        tax_evasion = any(re.search(rf"\b{kw}\b", text.lower()) for kw in self.tax_evasion_keywords)
        
        return {
            "is_threat": threats_found or (ml_prediction == "threat"),
            "is_tax_fraud": tax_evasion,
            "keywords_found": [kw for kw in self.tax_evasion_keywords if kw in text.lower()],
            "sentiment": self.sia.polarity_scores(text)
        }

print("Creating detector instance...")  # Debug print
detector = FraudDetector()

print("Creating UI widgets...")  # Debug print
text_input = widgets.Textarea(
    value='',
    placeholder='Try: "Pay me cash to avoid taxes"',
    description='Message:',
    layout={'width': '500px', 'height': '150px'},
    disabled=False
)

analyze_button = widgets.Button(
    description='Analyze for Fraud',
    button_style='danger',
    tooltip='Click to analyze',
    icon='search'
)

output = widgets.Output()

def on_button_click(b):
    with output:
        clear_output()
        user_text = text_input.value.strip()
        print("Button clicked! Processing...")  # Debug print
        
        if not user_text:
            print("⚠️ Please enter a message first!")
            return
            
        try:
            analysis = detector.analyze_message(user_text)
            print("🔍 Analysis Results:")
            print("-" * 40)
            
            if analysis['is_threat']:
                print("🚨 THREAT DETECTED!")
                print("This message contains threatening language")
                
            if analysis['is_tax_fraud']:
                print("💰 TAX FRAUD SUSPECTED")
                print(f"Keywords found: {', '.join(analysis['keywords_found'])}")
                
            if not (analysis['is_threat'] or analysis['is_tax_fraud']):
                print("✅ No fraud or threats detected")
                print("This message appears safe")
            
            print("\nSentiment Analysis:")
            print(f"Polarity score: {analysis['sentiment']['compound']:.2f} "
                 f"({'positive' if analysis['sentiment']['compound'] > 0 else 'negative' if analysis['sentiment']['compound'] < 0 else 'neutral'})")
        
        except Exception as e:
            print(f"❌ Error: {str(e)}")

print("Attaching button handler...")  # Debug print
analyze_button.on_click(on_button_click)

print("Displaying UI...")  # Debug print
display(widgets.VBox([
    widgets.HTML("<h1 style='color:red;'>Fraud Detection System</h1>"),
    widgets.HTML("<p>Enter a message to analyze for threats or tax fraud</p>"),
    text_input,
    analyze_button,
    output
]))

print("System ready! Try entering a message and clicking the button.")  # Debug print

Creating detector instance...
Initializing fraud detector...
Training model...
Model trained successfully!
Creating UI widgets...
Attaching button handler...
Displaying UI...


System ready! Try entering a message and clicking the button.
